In [1]:
#pip install apache-beam
!pip install apache-beam[interactive]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.7/14.7 MB 77.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.7/89.7 kB 10.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.7/138.7 kB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.0/152.0 kB 14.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 63.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.4/43.4 kB 4.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 671.3/671.3 kB 42.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 83.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.4/300.4 kB 25.4 MB/s eta 0:00:00
  Created wheel for crcmod: filename=crcmod-1.7-cp310-cp310-linux_x86_64.whl size=31407 sha256=e7203c6282639b2b9

In [ ]:
!mkdir -p data
!gsutil cp gs://dataflow-samples/shakespeare/kinglear.txt data/

Copying gs://dataflow-samples/shakespeare/kinglear.txt...
/ [1 files][153.6 KiB/153.6 KiB]                                                
Operation completed over 1 objects/153.6 KiB.                                    


In [ ]:
import apache_beam as beam
import re

inputs_pattern = 'data/*'
outputs_prefix = 'outputs/part'

def takeSecond(elem):
    return elem[1]

# Running locally in the DirectRunner.
with beam.Pipeline() as pipeline:
  (
      pipeline
      | 'Read lines' >> beam.io.ReadFromText(inputs_pattern)
      | 'Find words' >> beam.FlatMap(lambda line: re.findall(r"[a-zA-Z]+", line))
      | 'Pair words with 1' >> beam.Map(lambda word: (word, 1))
      | 'Group and sum' >> beam.CombinePerKey(sum)
      | 'sort descending' >> beam.combiners.Top.Of(20,key=takeSecond,reverse=False)
      | 'Format results' >> beam.Map(lambda word_count: str(word_count))
      | 'Write results' >> beam.io.WriteToText(outputs_prefix)
  )

In [2]:
import apache_beam as beam
import re
import requests


inputs_pattern = 'data/*'
outputs_prefix = 'outputs/part'


def get_data():
  url = "https://imdb-top-100-movies.p.rapidapi.com/"
  headers = {
	"X-RapidAPI-Key": "11e969206dmsh2dc567692521a17p1aff25jsnf8d1e65d23f9",
	"X-RapidAPI-Host": "imdb-top-100-movies.p.rapidapi.com"}

  response = requests.request("GET", url, headers=headers)
  return list(response.json())


class json_to_txt(beam.DoFn):
  def __init__(self):
    pass

  def process(self, movie_json):
    values_list = list(movie_json.values())
    list_output = list(map(str, values_list))
    movie_str = '|'.join(list_output)
    yield movie_str


# Running locally in the DirectRunner.
with beam.Pipeline() as pipeline:
  (
      pipeline
      | 'Read Data From API' >> beam.Create(get_data())
      | 'Convert json to CSV' >> beam.ParDo(json_to_txt())
      | 'Write results' >> beam.io.WriteToText(outputs_prefix)
  )


In [ ]:
data =get_data()
columns = ['rank', 'title', 'thumbnail', 'rating', 'id', 'year', 'image', 'description', 'trailer','imdbid']

In [ ]:
columns = ['rank', 'title', 'thumbnail', 'rating', 'id', 'year', 'image', 'description', 'trailer','imdbid']

for a in data[0:1]:
  values_list = list(a.values())
  list_output = list(map(str, values_list))
  #print(list_output)
  print('|'.join(list_output))

1|The Shawshank Redemption|https://m.media-amazon.com/images/M/MV5BMDFkYTc0MGEtZmNhMC00ZDIzLWFmNTEtODM1ZmRlYWMwMWFmXkEyXkFqcGdeQXVyMTMxODk2OTU@._V1_UY67_CR0,0,45,67_AL_.jpg|9.3|top1|1994|https://m.media-amazon.com/images/M/MV5BMDFkYTc0MGEtZmNhMC00ZDIzLWFmNTEtODM1ZmRlYWMwMWFmXkEyXkFqcGdeQXVyMTMxODk2OTU@._V1_QL75_UX380_CR0,1,380,562_.jpg|Two imprisoned men bond over a number of years, finding solace and eventual redemption through acts of common decency.|https://www.youtube.com/embed/NmzuHjWmXOc|['Drama']|['Frank Darabont']|['Stephen King (based on the short novel "Rita Hayworth and the Shawshank Redemption" by)', 'Frank Darabont (screenplay by)']|tt0111161
